# Loading Libraries  

In [96]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

# Declaring the directory for the script and loading the datasets

In [97]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin" or platform.system() == "Linux":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    if(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")




Username:  davidguzman


# Creating the sales table using joins 

In [98]:
del dfVendite['Unnamed: 0']
dfVendite.head()

,NrMovimento,budget/cons,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA
0,21621,BUDGET,ART0001433,C00140,4,18.80
1,21629,BUDGET,ART0001610,C00140,12,145.28
2,21631,BUDGET,ART0003269,C00140,24,933.99
3,21633,BUDGET,ART0002249,C00140,1,36.33
4,21634,BUDGET,ART0002370,C00140,1,65.72


In [99]:
del dfClienti['Unnamed: 0']
del dfClienti['CodCondizioniPagam']
del dfClienti['FattCumulative']

dfClienti.head()

,Nr,Valuta
0,C00001,1
1,C00003,2
2,C00004,3
3,C00005,1
4,C00006,2


In [100]:
del dfCambio['Unnamed: 0']
dfCambio.head()

,CodiceValuta,Anno,TassoCambioMedio
0,1,BUDGET,1
1,2,BUDGET,"1,0541"
2,3,BUDGET,123.4
3,1,CONSUNTIVO,1
4,2,CONSUNTIVO,"1,1993"


## Join operations for the sales table

In [101]:
temp = sqldf(
    "SELECT  * FROM dfVendite ca JOIN dfClienti cl ON ca.NrOrigine = cl.Nr ")
del temp["Nr"]
temp.columns = ['NrMovimento', 'budget', 'NrArticolo', 'NrOrigine',
                'Quantity', 'ImportoVenditaValutaLocaleTOTALEVENDITA', 'Valuta']

# Renaming the variables 

temp.loc[temp.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
temp.loc[temp.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"

tot = sqldf(
    "SELECT * FROM temp t  JOIN dfCambio c ON t.Valuta = c.CodiceValuta AND t.budget = c.Anno ")

del tot["Valuta"]
del tot["Anno"]

tot.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


## Sales table for the final statement

In [102]:
salesFinal = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Consuntivo' OR t.budget = 'CONSUNTIVO'")
salesFinal.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,35089,CONSUNTIVO,ART0000814,C00140,1,545.20,2,"1,1993"
1,35550,CONSUNTIVO,ART0000841,C00140,1,4650.13,2,"1,1993"
2,35894,CONSUNTIVO,ART0005136,C00483,1,1550.00,2,"1,1993"
3,35944,CONSUNTIVO,ART0000752,C00140,2,912.00,2,"1,1993"
4,35945,CONSUNTIVO,ART0000782,C00140,1,295.00,2,"1,1993"


## Sales table for the budget 

In [103]:
salesBudget = sqldf(
    "SELECT * FROM tot t WHERE t.budget = 'Budget' OR t.budget = 'BUDGET'")
salesBudget.head()


,NrMovimento,budget,NrArticolo,NrOrigine,Quantity,ImportoVenditaValutaLocaleTOTALEVENDITA,CodiceValuta,TassoCambioMedio
0,21621,BUDGET,ART0001433,C00140,4,18.80,2,"1,0541"
1,21629,BUDGET,ART0001610,C00140,12,145.28,2,"1,0541"
2,21631,BUDGET,ART0003269,C00140,24,933.99,2,"1,0541"
3,21633,BUDGET,ART0002249,C00140,1,36.33,2,"1,0541"
4,21634,BUDGET,ART0002370,C00140,1,65.72,2,"1,0541"


# Creating the costs table using joins

In [104]:
del dfConsumi["Unnamed: 0"]

dfConsumi.columns = ['NrMovimento', 'budget', 'CodiceMP', 'NrArticolo',
                     'NrDocumento', 'QuantityMPImpiegata', 'ImportoCostoTOTALE']

dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Consuntivo", "budget"] = "CONSUNTIVO"
dfConsumi.loc[dfConsumi.loc[:, "budget"] == "Budget", "budget"] = "BUDGET"
dfConsumi.head()

,NrMovimento,budget,CodiceMP,NrArticolo,NrDocumento,QuantityMPImpiegata,ImportoCostoTOTALE
0,28854,BUDGET,ART0001074,ART0000018,ODP17-05342,30.0,5.15
1,66819,CONSUNTIVO,ART0001074,ART0000018,ODP19-05998,30.0,5.15
2,28853,BUDGET,ART0005696,ART0000041,ODP17-05342,4.0,92.00
3,66818,CONSUNTIVO,ART0005696,ART0000041,ODP19-05998,4.0,88.00
4,28855,BUDGET,ART0005697,ART0000042,ODP17-05342,4.0,104.00


In [105]:
#Costo orario consuntivo
del dfCostoOrarioConsuntivo["Unnamed: 0"]
dfCostoOrarioConsuntivo


,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,60
1,RIS1,A11,60
2,RIS1,A20,80
3,RIS1,A30,80
4,RIS1,A32,60
...,...,...,...
65,RIS9,A11,50
66,RIS9,A20,80
67,RIS9,A30,50
68,RIS9,A32,50


In [106]:
#Budget
del dfCostoOrarioBudget["Unnamed: 0"]
dfCostoOrarioBudget.head()

,Risorsa,AreaProduzione,CostoOrario
0,RIS1,A10,51.0
1,RIS1,A11,51.0
2,RIS1,A20,68.0
3,RIS1,A30,68.0
4,RIS1,A32,51.0


In [107]:
del dfImpiegoRisorse["Unnamed: 0"]
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Consuntivo", "budgetConsuntivo"] = "CONSUNTIVO"
dfImpiegoRisorse.loc[dfImpiegoRisorse.loc[:, "budgetConsuntivo"] == "Budget", "budgetConsuntivo"] = "BUDGET"
dfImpiegoRisorse.head()

,NrArticolo,budgetConsuntivo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,QuantitydiOutput
0,ART0000018,BUDGET,ODP17-05342,Tornitura,A20,RIS2,1.00,2.0
1,ART0000018,BUDGET,ODP17-05342,Fresatura,A30,RIS7,0.75,2.0
2,ART0000018,BUDGET,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,2.0
3,ART0000018,CONSUNTIVO,ODP19-05998,Tornitura,A20,RIS2,1.00,2.0
4,ART0000018,CONSUNTIVO,ODP19-05998,Fresatura,A30,RIS5,1.00,2.0


## Costs table for the final statement

In [108]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'CONSUNTIVO'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'CONSUNTIVO'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totConsuntivo = sqldf("""SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP
FROM tot AS t JOIN dfCostoOrarioConsuntivo AS c on t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")


totConsuntivo.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP19-05998,Tornitura,A20,RIS2,1.0000,80,2.0,66819,ART0001074,30.0,5.15
1,ART0000018,ODP19-05998,Fresatura,A30,RIS5,1.0000,50,2.0,66819,ART0001074,30.0,5.15
2,ART0000018,ODP19-05998,Controllo qualità,CQ,RIS16,0.0000,15,2.0,66819,ART0001074,30.0,5.15
3,ART0000041,ODP19-05998,Tornitura,A20,RIS1,3.0000,80,4.0,66818,ART0005696,4.0,88.00
4,ART0000041,ODP19-05998,Fresatura,A30,RIS1,0.0001,80,4.0,66818,ART0005696,4.0,88.00


## Costs table for the budget

In [109]:
a = sqldf("SELECT * FROM dfImpiegoRisorse WHERE budgetConsuntivo = 'BUDGET'")

b = sqldf("SELECT * FROM dfConsumi WHERE budget = 'BUDGET'")

tot = sqldf("SELECT distinct * FROM a AS a join b AS b ON a.NrArticolo = b.NrArticolo and a.NrOrdineProduzione = b.NrDocumento")

totBudget = sqldf(
    """SELECT distinct NrArticolo, NrOrdineProduzione, Descrizione, NrAreaProduzione, t.Risorsa, TempoRisorsa, CostoOrario as CostoOrarioRisorsa, QuantitydiOutput, 
NrMovimento as NrMovimentoMP, CodiceMP, QuantityMPImpiegata, ImportoCostoTOTALE as ImportoCostoTotaleMP 
    FROM tot AS t JOIN  dfCostoOrarioBudget AS c ON t.Risorsa = c.Risorsa and t.NrAreaProduzione = c.AreaProduzione""")

totBudget.head()


,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,TempoRisorsa,CostoOrarioRisorsa,QuantitydiOutput,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP
0,ART0000018,ODP17-05342,Tornitura,A20,RIS2,1.00,68.00,2.0,28854,ART0001074,30.0,5.15
1,ART0000018,ODP17-05342,Fresatura,A30,RIS7,0.75,51.00,2.0,28854,ART0001074,30.0,5.15
2,ART0000018,ODP17-05342,Controllo qualità,CQ,RIS16,0.00,12.75,2.0,28854,ART0001074,30.0,5.15
3,ART0000041,ODP17-05342,Tornitura,A20,RIS14,3.00,51.00,4.0,28853,ART0005696,4.0,92.00
4,ART0000041,ODP17-05342,Tornitura,A20,RIS14,1.50,51.00,0.0,28853,ART0005696,4.0,92.00


# Scostamento costi di materie prime

costo = volume produttivo * sum (impiego i-esimo * costo acquisizone i-esimo) 

che corriponde a 

costo = QuantitydiOutput * sum((ImportoCostoTotale/QuantityMPImpiegata) * QuantityMPImpiegata)

## Scelgiere come raggruppare

In [110]:
CostiBudgetMP = sqldf('''select distinct NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,sum(TempoRisorsa), sum(QuantitydiOutput) 
from totBudget group by NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP order by sum(QuantitydiOutput) desc''')

CostiBudgetMP

,NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrarioRisorsa,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,sum(TempoRisorsa),sum(QuantitydiOutput)
0,ART0003610,ODP17-06015,Controllo qualità,CQ,RIS16,12.75,28010,ART0005335,189.0,28.35,0.00,180.0
1,ART0003610,ODP17-06015,Controllo qualità,CQ,RIS16,12.75,31153,ART0005336,180.0,144.00,0.00,180.0
2,ART0003610,ODP17-06015,Controllo qualità,CQ,RIS16,12.75,31154,ART0005154,9000.0,489.14,0.00,180.0
3,ART0004363,ODP17-15962,Tornitura,A20,RIS2,68.00,35504,ART0005953,116.0,826.57,4.20,116.0
4,ART0004384,ODP17-15356,Controllo qualità,CQ,RIS16,12.75,34986,ART0005933,110.0,330.00,0.00,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3481,ART0005136,ODP17-04538,Montaggio,A40,RIS4,42.50,26637,ART0001807,1.0,52.35,0.25,0.0
3482,ART0005136,ODP17-04538,Montaggio,A40,RIS4,42.50,26638,ART0003839,2.0,10.00,0.25,0.0
3483,ART0005136,ODP17-04557,Tornitura,A20,RIS2,68.00,26645,ART0005176,42.0,0.00,3.25,0.0
3484,ART0005136,ODP17-04557,Tornitura,A20,RIS2,68.00,26646,ART0005176,12.0,0.00,3.25,0.0


In [111]:
CostiBudgetMP = sqldf('''select distinct NrArticolo,NrOrdineProduzione,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP, sum(QuantitydiOutput) as QuantitydiOutput
from totBudget group by NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP order by sum(QuantitydiOutput) desc''')

CostiBudgetMP

,NrArticolo,NrOrdineProduzione,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,QuantitydiOutput
0,ART0003610,ODP17-06015,28010,ART0005335,189.0,28.35,180.0
1,ART0003610,ODP17-06015,31153,ART0005336,180.0,144.00,180.0
2,ART0003610,ODP17-06015,31154,ART0005154,9000.0,489.14,180.0
3,ART0004363,ODP17-15962,35504,ART0005953,116.0,826.57,116.0
4,ART0004384,ODP17-15356,34986,ART0005933,110.0,330.00,110.0
...,...,...,...,...,...,...,...
1860,ART0005136,ODP17-04538,26637,ART0001807,1.0,52.35,0.0
1861,ART0005136,ODP17-04538,26638,ART0003839,2.0,10.00,0.0
1862,ART0005136,ODP17-04557,26645,ART0005176,42.0,0.00,0.0
1863,ART0005136,ODP17-04557,26646,ART0005176,12.0,0.00,0.0


## Budget standard

In [112]:
a = sqldf(''' SELECT DISTINCT NrArticolo, NrOrdineProduzione, QuantitydiOutput
              FROM  CostiBudgetMP
              ORDER BY NrArticolo
              ''')
a

,NrArticolo,NrOrdineProduzione,QuantitydiOutput
0,ART0000018,ODP17-05342,2.00
1,ART0000041,ODP17-05342,4.00
2,ART0000042,ODP17-05342,4.00
3,ART0000043,ODP17-05342,2.25
4,ART0000043,ODP17-05342,2.00
...,...,...,...
964,ART0005136,ODP17-16230,1.00
965,ART0005136,ODP17-00667,0.00
966,ART0005136,ODP17-04538,0.00
967,ART0005136,ODP17-04557,0.00


In [113]:
b = sqldf(''' SELECT  NrArticolo, NrOrdineProduzione, SUM(ImportoCostoTotaleMP/QuantityMPImpiegata * QuantityMPImpiegata) AS ImportoCostoTotaleMP
              FROM  CostiBudgetMP
              GROUP BY NrArticolo
              ''')
b

,NrArticolo,NrOrdineProduzione,ImportoCostoTotaleMP
0,ART0000018,ODP17-05342,5.15
1,ART0000041,ODP17-05342,92.00
2,ART0000042,ODP17-05342,104.00
3,ART0000043,ODP17-05342,72.12
4,ART0000115,ODP17-04831,434.44
...,...,...,...
499,ART0005109,ODP17-08620,6.97
500,ART0005110,ODP17-09406,74.00
501,ART0005111,ODP17-09406,16.00
502,ART0005112,ODP17-09406,26.00


In [114]:
cBudget = sqldf(''' SELECT  A.NrArticolo, A. NrOrdineProduzione, A.QuantitydiOutput * B.ImportoCostoTotaleMP as costoMP
              FROM  a as A JOIN b as B
              ON A.NrArticolo = B.NrArticolo
              AND A.NrOrdineProduzione = B.NrOrdineProduzione
              GROUP BY A.NrArticolo, A. NrOrdineProduzione
              ''')
cBudget


,NrArticolo,NrOrdineProduzione,costoMP
0,ART0000018,ODP17-05342,10.30
1,ART0000041,ODP17-05342,368.00
2,ART0000042,ODP17-05342,416.00
3,ART0000043,ODP17-05342,162.27
4,ART0000115,ODP17-04831,1303.32
...,...,...,...
499,ART0005109,ODP17-08620,27.88
500,ART0005110,ODP17-09406,148.00
501,ART0005111,ODP17-09406,32.00
502,ART0005112,ODP17-09406,52.00


In [115]:
sqldf('SELECT NrArticolo, count(*) FROM cBudget group by NrArticolo order by count(*)') # are they unique

,NrArticolo,count(*)
0,ART0000018,1
1,ART0000041,1
2,ART0000042,1
3,ART0000043,1
4,ART0000115,1
...,...,...
499,ART0005109,1
500,ART0005110,1
501,ART0005111,1
502,ART0005112,1


## Budget effettivo

In [116]:
CostiConsuntivoMP = sqldf('''select distinct NrArticolo,NrOrdineProduzione,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP, sum(QuantitydiOutput) as QuantitydiOutput
from totConsuntivo group by NrArticolo,NrOrdineProduzione,Descrizione,NrAreaProduzione,Risorsa,CostoOrarioRisorsa,
NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP order by sum(QuantitydiOutput) desc''')

CostiConsuntivoMP

,NrArticolo,NrOrdineProduzione,NrMovimentoMP,CodiceMP,QuantityMPImpiegata,ImportoCostoTotaleMP,QuantitydiOutput
0,ART0004889,ODP18-06579,48148,ART0005867,43.0,361.93,258.0
1,ART0004889,ODP18-06579,48149,ART0005186,2451.0,102.89,258.0
2,ART0005136,ODP18-05973,45951,ART0005176,40.0,0.00,142.0
3,ART0005136,ODP18-05973,45952,ART0005176,102.0,0.00,142.0
4,ART0004889,ODP18-04217,42056,ART0005867,62.0,569.97,124.0
...,...,...,...,...,...,...,...
3223,ART0005136,ODP18-08241,50136,ART0005176,1.0,0.00,0.0
3224,ART0005136,ODP19-01422,54118,ART0005176,4.0,0.00,0.0
3225,ART0005136,ODP19-01954,56250,ART0005176,1.0,0.00,0.0
3226,ART0005136,ODP19-02530,57669,ART0005176,18.0,0.00,0.0


In [117]:
a = sqldf(''' SELECT DISTINCT NrArticolo, NrOrdineProduzione, QuantitydiOutput
              FROM  CostiConsuntivoMP
              ORDER BY NrArticolo
              ''')
b = sqldf(''' SELECT  NrArticolo, NrOrdineProduzione, SUM(ImportoCostoTotaleMP/QuantityMPImpiegata * QuantityMPImpiegata) AS ImportoCostoTotaleMP
              FROM  CostiConsuntivoMP
              GROUP BY NrArticolo
              ''')
cConsuntivo = sqldf(''' SELECT  A.NrArticolo, A. NrOrdineProduzione, A.QuantitydiOutput * B.ImportoCostoTotaleMP as costoMP
              FROM  a as A JOIN b as B
              ON A.NrArticolo = B.NrArticolo
              AND A.NrOrdineProduzione = B.NrOrdineProduzione
              GROUP BY A.NrArticolo, A. NrOrdineProduzione
              ''')

In [118]:
a

,NrArticolo,NrOrdineProduzione,QuantitydiOutput
0,ART0000018,ODP19-05998,2.0
1,ART0000041,ODP19-05998,4.0
2,ART0000042,ODP19-05998,4.0
3,ART0000043,ODP19-05998,2.0
4,ART0000043,ODP19-05998,1.0
...,...,...,...
1472,ART0005136,ODP18-08241,0.0
1473,ART0005136,ODP19-01422,0.0
1474,ART0005136,ODP19-01954,0.0
1475,ART0005136,ODP19-02530,0.0


In [119]:
b

,NrArticolo,NrOrdineProduzione,ImportoCostoTotaleMP
0,ART0000018,ODP19-05998,5.15
1,ART0000041,ODP19-05998,88.00
2,ART0000042,ODP19-05998,100.00
3,ART0000043,ODP19-05998,107.13
4,ART0000115,ODP19-00633,268.26
...,...,...,...
499,ART0005109,ODP18-06907,14.74
500,ART0005110,ODP18-06907,378.00
501,ART0005111,ODP18-06907,96.00
502,ART0005112,ODP18-06907,104.00


In [120]:
cConsuntivo

,NrArticolo,NrOrdineProduzione,costoMP
0,ART0000018,ODP19-05998,10.30
1,ART0000041,ODP19-05998,352.00
2,ART0000042,ODP19-05998,400.00
3,ART0000043,ODP19-05998,214.26
4,ART0000115,ODP19-00633,804.78
...,...,...,...
499,ART0005109,ODP18-06907,88.44
500,ART0005110,ODP18-06907,1512.00
501,ART0005111,ODP18-06907,384.00
502,ART0005112,ODP18-06907,416.00


In [121]:
sqldf('SELECT NrArticolo, count(*) FROM cConsuntivo group by NrArticolo order by count(*)') # are they unique

,NrArticolo,count(*)
0,ART0000018,1
1,ART0000041,1
2,ART0000042,1
3,ART0000043,1
4,ART0000115,1
...,...,...
499,ART0005109,1
500,ART0005110,1
501,ART0005111,1
502,ART0005112,1


In [122]:
sqldf(''' SELECT a.NrArticolo, a.costoMP, b.CostoMP, a.costoMP-b.costoMP
FROM cBudget as a join cConsuntivo as b
ON a.NrArticolo = b.NrArticolo
''')

,NrArticolo,costoMP,costoMP,a.costoMP-b.costoMP
0,ART0000018,10.30,10.30,0.00
1,ART0000041,368.00,352.00,16.00
2,ART0000042,416.00,400.00,16.00
3,ART0000043,162.27,214.26,-51.99
4,ART0000115,1303.32,804.78,498.54
...,...,...,...,...
499,ART0005109,27.88,88.44,-60.56
500,ART0005110,148.00,1512.00,-1364.00
501,ART0005111,32.00,384.00,-352.00
502,ART0005112,52.00,416.00,-364.00


# Scostamento costi di Risorsa

## Diversi tipi risorse

In [123]:
sqldf('select distinct Descrizione from totConsuntivo order by Descrizione')

,Descrizione
0,None
1,Brocciatura / stozzatrice
2,Controllo qualità
3,Fresatura
4,Fresatura (sbavatura fori)
5,Fresatura (sistemazione ore)
6,Montaggio
7,Piegatura
8,Preparazione materiale
9,Preparazione materiale/taglio/sbavatura
